# Rote-Learning Training

In [ ]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

Der Rote-Learning Algorithmus muss im wahrsten Sinne des Wortes auswendig lernen und das Spiel trainieren. Dazu werden möglichst viele Spiele hintereinander gespielt und die Transpositionstabelle mit immer mehr und genaueren Daten gefüllt.

Zunächst werden beide implementierte Algorithmen geladen: `AlphaBetaPruning` und `Minimax`. Die `Match` Implementierung aus dem `Tournament` Notebook kann ebenfalls wieder verwendet werden.

In [ ]:
%run ./nmm-alpha-beta-pruning.ipynb
%run ./nmm-minimax.ipynb
%run ./nmm-tournament.ipynb

Um eine übersichtlichere Entwicklung zu ermöglichen, werden Typdefinitionen geladen, welche später im Code verwendet werden. Das Paket `tqdm` ermöglicht eine einfache Fortschrittsanzeige.

In [ ]:
from typing import Optional, Union, List, Callable
from tqdm.notebook import tqdm

## Training

Innerhalb eines Trainings spielt eine künstliche Intelligenz viele Male gegen sich selbst und erweitert so ihre Transpositionstabelle. Bei jedem Spiel gegen sich selbst ist eine höhere Rekursionstiefe erreichbar, da der Cache bereits Werte von vorherigen Runden beinhaltet. Da die Implementierung der Transpositionstablle die gespeicherten Werte auf den weißen Spieler normiert, ist es sogar effizient möglich, dass sich beide künstliche Intelligenzen (weiß und schwarz) einen Cache teilen.

Innerhalb eines Trainings werden nur die ersten 18 Runden eines jeden Spieles gespielt, da alle diese Runden gezwungenermaßen in der ersten Spielphase `Placing` sind. Jeder Spieler muss alle seine 9 Steine zu Beginn der Runde setzen. Dadurch werden innerhalb der 18 Runden alle Steine von beiden Spielern gesetzt. Es wird nur die Spielphase `Placing` trainiert, da diese Phase im Vergleich zur zweiten Phase `Moving` um einiges komplexer ist. Bei jedem Zug kann jeder neue Stein eines Spielers auf jedes freie Feld bewegt werden, in der `Moving` Phase sind hingengen nur benachbarte leere Felder möglich. Dadurch ist der Suchraum für die Algorithmen zu Beginn des Spieles besonders groß und die künstliche Intelligenz kann dementsprechend weniger Züge in die Zukunft schauen. Durch das Auswendiglernen dieser Phase gewinnen die Algorithmen einen größeren Vorteil.

Auch die letzte Phase `Flying` ist sehr komplex und würde vom Auswendiglernen profitieren, allerdings wird hierzu eine Endspieldatenbank implementiert. Diese Datenbank deckt die letzte Phase besser ab als das Auswendiglernen.

Die Klasse `Training` implementiert solch ein Training und benötigt mehrere Parameter:

Verpflichtend:
* `cache` ist die Instanz der Transpositionstabelle, die innerhalb des Trainings verwendet und damit trainiert werden soll;
* `artificial_intelligence` ist eine Funktion die eine `ArtificialIntelligence` Instanz produziert, die jeweils für den weißen und den schwarzen Spieler aufgerufen wird;

Optional:
* `trainings` ist die Anzahl der Runden in denen trainiert werden sollen;
* `seed_offset` ist die Zahl, die auf den Seed addiert werden soll um verschiedene Trainingsläufe erstellen zu können;
* `path_prefix` ist der Prefix der vor den Dateinamen geschrieben wird, um den Cache Zwischenstand zu speichern;
* `save_interval` ist das Intervall in dem der Cache auf der Festplatte zwischen gespeichert werden soll.

In [ ]:
class Training():
    def __init__(
        self,
        cache: Cache,
        artificial_intelligence: Callable[[Cache], ArtificialIntelligence],
        trainings: int = 100,
        seed_offset: int = 0,
        path_prefix: str = "training-",
        save_interval: int = 10
    ):
        self.cache = cache
        self.artificial_intelligence = artificial_intelligence
        self.trainings = trainings
        self.seed_offset = seed_offset
        self.path_prefix = path_prefix
        self.save_interval = save_interval

Für Entwicklungszwecke wird eine Stringdarstellung für die Klasse `Training` implementiert. Hierzu wird durch die Funktion `__repr__` ein String zurückgegeben, der alle Parameter der Klasse beinhaltet.

In [ ]:
def __repr__(self: Training):
    return f"Training(cache={self.cache}, " + \
           f"artificial_intelligence={type(self.artificial_intelligence).__name__}, " + \
           f"trainings={self.trainings}, seed_offset={self.seed_offset}, " + \
           f"path_prefix='{self.path_prefix}', save_interval={self.save_interval})"

Training.__repr__ = __repr__
del __repr__

Die Funktion `train` trainiert die gespeicherte Transpositionstabelle indem für die Anzahl der zu spielenden Runden (`trainings`) ein Spiel (`Match`) erstellt wird. Dieses Spiel wird von zwei `ArtificialIntelligence` Instanzen gespielt, welche durch die gespeicherte Funktion `artificial_intelligence` erstellt werden. Pro Spiel werden maximal 18 Runden, also nur die `Placing` Phase gespielt. Nach `save_interval` Spielen wird die Transpositionstabelle mit der Rundennummer auf der Festplatte zwischengespeichert.

In [ ]:
def train(self: Training):
    for i in tqdm(range(self.trainings)):
        name = f"{self.path_prefix}{i+1:0>3}"
        random.seed(self.seed_offset + i)
        match = Match(
            white     = self.artificial_intelligence(cache = self.cache),
            black     = self.artificial_intelligence(cache = self.cache),
            max_turns = 18,
            name      = name
        )
        
        print(f"Training #{name}:")
        match.play()
        print(f" > {self.cache}")
        
        if (i+1) % self.save_interval == 0:
            print(f"> Saving to '{name}.cache'...")
            self.cache.save(f"{name}.cache")
        print()
    self.cache.save(f"{self.path_prefix}final.cache")

Training.train = train
del train